# Infonet Security HW4
### Harris A. Ransom
### 11/20/2024

In [43]:
# Imports
import numpy as np
import secrets
import hashlib
from math import gcd
import sympy
import time

## Helper Functions

In [44]:
# Extended Euclidean Algorithm
def gcdExtended(a, b):
    # Base Case
    if a == 0:
        return b, 0, 1
    
    gcd, x1, y1 = gcdExtended(b % a, a)
        
    # Update x and y using results of recursive call
    x = y1 - (b//a) * x1
    y = x1
    return gcd, x, y

# Implements modular inverse finder
def inverse_finder(a, n):
    isCoprime = (gcd(a,n) == 1)
    if (not isCoprime):
        raise ValueError("a and n are not coprime!")
    _, x, y = gcdExtended(a, n)
    result = (x % n + n) % n
    return result

# Implements efficient modular exponentiation algorithm
# returns (m ** e) % n
def exponentiation(m, e, n):
    r = 1 # Initialize result to 1
    e_bin = bin(e)[2:]
    for i in e_bin:
        r = (r**2) % n
        if (i == '1'):
            r = (r*m) % n
    return r

# Helper function to compute a SHA3-256 hash of a message
def shaHash(m):
    # Create SHA3-256 hash object
    h = hashlib.sha3_256()

    # Encode m
    m = str(m).encode()

    # Return hash of m
    h.update(m)
    return h.hexdigest()

## Q1: Secure Key Exchange

### Q1a: Diffie Hellman

In [45]:
# Performs Diffie Hellman key exchange computations
def DiffieHellman(alpha, p):
    # Alice
    x = secrets.randbits(256)
    A = exponentiation(alpha, x, p)
    
    # Bob
    y = secrets.randbits(256)
    B = exponentiation(alpha, y, p)
    
    # Alice receives B and computes shared secret
    Ka = exponentiation(B, x, p)
    
    # Bob receives A and computes shared secret
    Kb = exponentiation(A, y, p)
    
    # Outputs
    print(f"Alice's Secret Integer (x): {x}")
    print(f"Bob's Secret Integer (y): {y}")
    print(f"Alice's Message (A): {A}")
    print(f"Bob's Message (B): {B}")
    print(f"Alice's Computed Key (Ka): {Ka}")
    print(f"Bob's Computed Key (Kb): {Kb}")
    if (Ka == Kb):
        print("Computed keys match!")
        K = Ka
    else:
        print("Error: Computed key mismatch!")
        K = -1
    return A, B, K

alpha = 2
prime = 102188617217178804476387977160129334431745945009730065519337094992129677228373
A, B, K = DiffieHellman(alpha, prime)

Alice's Secret Integer (x): 22154383128226360424108748540258917568795639377044322853762896826702272548794
Bob's Secret Integer (y): 114076194838391609757540358024891046828944644658141348300488935902832966500786
Alice's Message (A): 100711379082281706897595169077822340133322185988097171620941867984902423472785
Bob's Message (B): 93024941944536752664109929246921171931182924937435024415241039521585661533448
Alice's Computed Key (Ka): 32071547943736115665050403073669512495348730700298905644579461665461232772058
Bob's Computed Key (Kb): 32071547943736115665050403073669512495348730700298905644579461665461232772058
Computed keys match!


### Q1b: RSA

In [46]:
# TODO: Finish RSA Key Exchange
# Generate a random prime number between a lower and upper bound
# See: https://stackoverflow.com/questions/21043075/generating-large-prime-numbers-in-python
def rand_prime(bits):
    while True:
        p = secrets.randbits(bits)
        if (sympy.isprime(p)):
            break
    return p

# RSA Key Generation
def RSA_key_generate(p, q, e=17):
    phi = (p-1)*(q-1)
    if (gcd(e, phi) != 1) or (e >= phi):
        raise ValueError("Improper selection of p, q, or e!")
    
    # Output/return keys
    n = p*q
    d = inverse_finder(e, phi)
    ke = (e, n)
    kd = (d, n)
    return ke, kd

# Performs a secure key exchange using the RSA cryptosystem
def RSA_KeyExchange(p, q):
    # Bob generates a random private key k
    k = secrets.randbelow(64) # k has to be small to fit block length
    print(f"Bob's private key k: {k}")
    
    # Alice generates her public/private key pair
    # and sends this information to Bob
    ke, kd = RSA_key_generate(p, q)

    # Bob encrypts k with ke
    e, n = ke
    c = exponentiation(k, e, n)
    print(f"Ciphertext c: {c}")
    
    # Bob sends ciphered key c to Alice
    # Alice decrypts c to recover k
    d, n, = kd
    k2 = exponentiation(c, d, n)
    print(f"Recovered key k2: {k2}")
    if (k2 == k):
        print("Keys match!")
        return True
    print("Key's don't match!")
    return False

p = 61
q = 53
exchangeSuccess = RSA_KeyExchange(p, q)

Bob's private key k: 2
Ciphertext c: 1752
Recovered key k2: 2
Keys match!


## Q2: Digital Signature Authentication

In [47]:
# ElGamal key generation function
# p: Sufficiently large prime number
# alpha: Primitive root mod p
def ElGamalKeygen(alpha, p):
    a = secrets.randbelow(p-2) + 1 # 1 <= a <= p-2
    beta = exponentiation(alpha, a, p) # Computes Beta for the public key
        # Public Key: (p, alpha, beta)
        # Private key: a
    pubKey = (p, alpha, beta)
    privKey = a
    return pubKey, privKey

# Implements signature generation for ElGamal authentication
def ElGamalSignatureGen(alpha, p, m):
    h = int(shaHash(m), 16) # Generate hash of message
    #print(f"Generated message hash: {h}")
    
    # Choose a secret k s.t. k and p-1 are coprime
    nums = np.arange(100, 100000)
    k = secrets.choice(nums)
    while (gcd(k, p-1) != 1):
        k = secrets.choice(nums)

    # Generate public and private keys
    pubKey, privKey = ElGamalKeygen(alpha, p)
    
    # Calculate r and s
    r = exponentiation(alpha, k, p)
    kInv = inverse_finder(k, p-1)
    s = (kInv * (h - (privKey*r))) % (p-1)

    return pubKey, privKey, r, s

# Verifies an ElGamal authentication signature
def ElGamalSigVerify(pubKey, r, s, m):
    p, alpha, beta = pubKey

    # Compute v1 and v2 values
    v1 = exponentiation(beta, r, p) * exponentiation(r, s, p)
    h2 = int(shaHash(m), 16)
    v2 = exponentiation(alpha, h2, p)
    #print(f"Recovered message hash: {h2}")

    # Verify signature
    if (((v1 - v2) % p) == 0):
        #print("Signature successfully verified!")
        return True
    else:
        #print("Signature not verified!")
        return False

# Implements ElGamal authentication scheme
def ElGamalAuth(alpha, p, m=123456, alterMessage=False):
    # Alice signature generation
    pubKey, privKey, r, s = ElGamalSignatureGen(alpha, p, m)
    print(f"Alice PubKey: {pubKey}")
    print(f"Alice PrivKey: {privKey}")
    
    # Alice sends Bob signature and message
    if (alterMessage):
        m = m ^ privKey
    verification = ElGamalSigVerify(pubKey, r, s, m)
    return verification
    
alpha = 2
p = 102188617217178804476387977160129334431745945009730065519337094992129677228373
goodAuth = ElGamalAuth(alpha, p, m=123456, alterMessage=False)
badAuth = ElGamalAuth(alpha, p, m=123456, alterMessage=True)
print(f"Unaltered Message Authentication Successful: {goodAuth}")
print(f"Altered Message Authentication Successful: {badAuth}")

Alice PubKey: (102188617217178804476387977160129334431745945009730065519337094992129677228373, 2, 80899835145357920942494557661460664162703362290248426297364679347580434985080)
Alice PrivKey: 34712516988083379691081339589397865502385108947735648349202094734018948174231
Alice PubKey: (102188617217178804476387977160129334431745945009730065519337094992129677228373, 2, 74865874635444876744646439324670460003019675083857352028775754047844475467536)
Alice PrivKey: 66113823857655645046645684943973332772381990883652895801214949166338344308767
Unaltered Message Authentication Successful: True
Altered Message Authentication Successful: False


## Q3: Replay Attacks

A **replay attack** is an attack in which an eavesdropper (Eve) records and *replays* information (e.g. hash, ticket, etc.) during a communication between two parties (Alice and Bob) in order to bypass security protocols. This attack is typically implemented as part of a **Man in the Middle (MitM)** attack. 

To protect against these kinds of attacks, secure communication protocols typically include some kind of nonce or timestamp. 
In this example, Alice and Bob perform a key exchange using RSA. Alice and Bob know each other's public keys, and the ciphertext they share is in the form of a token that can be used to "authenticate" themselves for a service. However, Eve is listening over their communication channel and intercepts the ciphertext of the token, which she then replays in order to access the service. In a non-timestamped cryptosystem, this should allow Eve to authenticate with Bob's service without having to attack the cryptography of the secret itself. In a timestamped cryptosystem, the timestamp feature embedded within the ciphertext should prevent Eve from being able to authenticate with Bob's service. 

In [48]:
# Helper function to get timestamp
def timestamp():
    return int(time.time()*100)

# RSA Encryption
def RSA_encrypt(m, ke):
    #msg = int.from_bytes(bytes(m, 'utf-8'), 'little')
    e, n = ke[0], ke[1]
    c = exponentiation(m, e, n)
    return c
    
# RSA Decryption
def RSA_decrypt(c, kd):
    d, n = kd[0], kd[1]
    msg = exponentiation(c, d, n)
    #msg = m.to_bytes((m.bit_length() + 7) // 8, 'little').decode('utf-8')
    return msg
    
# Demonstrate vulnerable non-timestamped cryptosystem
def VulnerableCryptosystem(keB, kdB, replayAttack=False):
    # Note: Alice and Bob perform secure key exchange beforehand to agree on token
    
    # Alice sends token to Bob
    token = 1234
    c = RSA_encrypt(token, keB)

    # Eve intercepts ciphered token
    token_eve = c

    # Bob receives Alice's original token and decrypts it with his private key
    token_bob = RSA_decrypt(c, kdB)
    if (token_bob == 1234):
        print("Successful authentication!")
    else:
        print("Authentication unsuccessful!")

    # If a replay attack is performed, Eve replays the recovered token to authenticate with Bob's service
    if (replayAttack):
        token_replay = RSA_decrypt(token_eve, kdB) # Bob receives ciphertext from Eve
        if (token_replay == 1234):
            print("Replay attack successful!")
        else:
            print("Replay attack unsuccessful!")

# Demonstrate improved timestamped cryptosystem
def TimestampedCryptosystem(keB, kdB, replayAttack=False):
    # Note: Alice and Bob perform secure key exchange beforehand to agree on token

    # Alice sends token with timestamp to Bob
    # TODO: Figure out how to add timestamp to cipher
    token = 1234
    c = RSA_encrypt(token, keB)

    # Eve intercepts ciphered token
    token_eve = c

    # Bob receives Alice's original token and decrypts it with his private key
    # TODO: Figure out how to decrypt ciphertext with timestamp
    token_bob = RSA_decrypt(c, kdB)
    if (token_bob == 1234):
        print("Successful authentication!")
    else:
        print("Authentication unsuccessful!")

    # If a replay attack is performed, Eve replays the recovered token to authenticate with Bob's service
    if (replayAttack):
        token_replay = RSA_decrypt(token_eve, kdB) # Bob receives ciphertext from Eve
        if (token_replay == 1234):
            print("Replay attack successful!")
        else:
            print("Replay attack unsuccessful!")

p = 61
q = 53 
keB, kdB = RSA_key_generate(p, q)
print("Performing non-replay experiment on vulnerable cryptosystem...")
VulnerableCryptosystem(keB, kdB, replayAttack=False)
print("Performing replay experiment on vulnerable cryptosystem...")
VulnerableCryptosystem(keB, kdB, replayAttack=True)
#print("Performing non-replay experiment on timestamped cryptosystem...")
#TimestampedCryptosystem(keB, kdB, replayAttack=False)
#print("Performing replay experiment on timestamped cryptosystem...")
#TimestampedCryptosystem(keB, kdB, replayAttack=True)
print("Finished experiments")

Performing non-replay experiment on vulnerable cryptosystem...
Successful authentication!
Performing replay experiment on vulnerable cryptosystem...
Successful authentication!
Replay attack successful!
Finished experiments
